In [ ]:
from __future__ import annotations

import os
import sys
import subprocess
import time
from pathlib import Path
from typing import Optional, Dict
import xml.etree.ElementTree as ET

# Bổ sung import TraCI
import traci

# ---------------------------------------------------------------------------
#  Project helpers và cấu hình
# ---------------------------------------------------------------------------

sys.path.append("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/src")

from simulationHelpers import od2tripsForSubs  # noqa: E402
from config import TLSCYCLEADAPTATION_PY, TLSCOORDINATOR_PY  # noqa: E402

# ---------------------------------------------------------------------------
#  Constants
# ---------------------------------------------------------------------------

THREADS = min(1, os.cpu_count() or 1)
os.environ["OMP_NUM_THREADS"] = str(THREADS)

NET_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml"
TAZ_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/taz.xml"
VTYPES_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes/eidm-1.xml"
ODS_DIR = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-old-form-all"
TRIPS_DIR = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-trips"
ROUTES_DIR = "home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-routes"

SIM_DIR = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/04-06-test-subs-strategy")
# TRIPS_DIR = SIM_DIR / "trips"
# ROUTES_DIR = SIM_DIR / "routes"
TLS_DIR = SIM_DIR / "tls"
OUTPUTS_DIR = SIM_DIR / "outputs"
STATES_DIR = SIM_DIR / "states"
LOGS_DIR = SIM_DIR / "logs"

trips = od2tripsForSubs(
    taz_file=TAZ_XML,
    trips_dir=TRIPS_DIR,
    od_dir=ODS_DIR,
    dist_id="car_eidm"
)
print("All generated trip files:")
for t in trips:
    print("  ", t)

In [ ]:
#!/usr/bin/env python3
from __future__ import annotations

import os
import sys
import subprocess
from pathlib import Path
from typing import Optional, Dict
import xml.etree.ElementTree as ET
from collections import defaultdict

# ---------------------------------------------------------------------------
#  Project helpers và cấu hình
# ---------------------------------------------------------------------------
sys.path.append("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/src")
from simulationHelpers import od2tripsForSubs  # noqa: E402

# ---------------------------------------------------------------------------
#  Constants (đã có trips)
# ---------------------------------------------------------------------------
NET_XML     = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml"
TAZ_XML     = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/taz.xml"
VTYPES_XML  = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes/eidm-1.xml"

ODS_DIR     = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-old-form-all"
TRIPS_DIR   = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-trips"
ROUTES_DIR  = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-routes"
SPLIT_DIR   = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/split-routes"

# Số threads cho duarouter
DUAROUTER_THREADS = "64"
# Seed (tuỳ ý)
SEED = 42

print("=== Step 1: Generating routes from each trip file ===")
os.makedirs(ROUTES_DIR, exist_ok=True)
routes_paths: list[str] = []

for trips_xml in sorted(Path(TRIPS_DIR).glob("trips_*.xml")):
    # Tạo tên file route tương ứng
    fname = trips_xml.name.replace("trips_", "routes_")
    route_xml = os.path.join(ROUTES_DIR, fname)
    duarouter_log = route_xml.replace(".xml", ".log")

    cmd = [
        "duarouter",
        "-n", NET_XML,
        "-r", str(trips_xml),
        "-a", VTYPES_XML,
        "-o", str(route_xml),
        "--log", str(duarouter_log),
        "--exit-times",
        "--named-routes",
        "--route-length",
        "--write-costs",
        "--routing-threads", DUAROUTER_THREADS,
        "--ignore-errors",
        "--seed", str(SEED),
    ]
    print(f"[RUN ] {' '.join(cmd)}")
    subprocess.run(cmd, check=True)
    routes_paths.append(route_xml)
    print(f"[INFO] Generated {route_xml}")
print()


In [ ]:
#!/usr/bin/env python3
"""
count_trips_per_hour.py
-----------------------
Đọc tất cả các file XML trong thư mục “smooth-routes” gốc (bỏ qua các file *.alt.xml),
đếm số <vehicle> trong mỗi file, rồi tổng hợp theo giờ (XX). Kết quả in ra console.

Cách hoạt động:
1. Quét folder SMOOTH_ROUTES_DIR, tìm các file “routes_XX_YofZ.xml” (không tính *.alt.xml).
2. Với mỗi file, parse XML và đếm số thẻ <vehicle>.
3. Lấy chỉ số giờ XX từ tên file, cộng dồn vào dict hour_to_count.
4. Cuối cùng in số chuyến (vehicles) theo từng giờ.

Cách chạy:
    chmod +x count_trips_per_hour.py
    ./count_trips_per_hour.py
"""

import sys
import re
import xml.etree.ElementTree as ET
from pathlib import Path
from collections import defaultdict

# ===================================================================
# Cấu hình: đường dẫn tới thư mục chứa các file route gốc (smooth-routes)
# ===================================================================
SMOOTH_ROUTES_DIR = Path(
    "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/"
    "data/processed-data/total_scenario/smooth-routes"
)

# ===================================================================
# Regex để bắt tên file: routes_XX_YofZ.xml (ví dụ "routes_07_2of3.xml")
# Bỏ qua mọi file chứa ".alt.xml" tự động
# ===================================================================
pattern = re.compile(r"^routes_(\d{2})_\d+of\d+\.xml$")

def count_trips_per_hour(routes_dir: Path) -> dict[str, int]:
    """
    Quét hết các file *.xml trong routes_dir (bỏ qua *.alt.xml),
    parse và đếm vehicle, rồi tổng hợp theo giờ.

    Trả về dict: key = "XX" (chuỗi 2 chữ số giờ), value = tổng số xe (int).
    """
    hour_to_count: dict[str, int] = defaultdict(int)

    for f in routes_dir.glob("*.xml"):
        if ".alt.xml" in f.name:
            continue
        m = pattern.match(f.name)
        if not m:
            continue

        hour = m.group(1)  # "07", "15", v.v.
        try:
            tree = ET.parse(f)
            root = tree.getroot()
        except ET.ParseError as e:
            print(f"[WARNING] Không parse được {f.name}: {e}", file=sys.stderr)
            continue

        # Đếm tất cả thẻ <vehicle> trực tiếp con của <routes>
        # Nếu có namespace, dùng .//vehicle để bắt mọi vehicle
        vehicles = root.findall(".//vehicle")
        count = len(vehicles)
        hour_to_count[hour] += count

    return dict(sorted(hour_to_count.items()))


def main():
    if not SMOOTH_ROUTES_DIR.is_dir():
        print(f"[ERROR] Thư mục không tồn tại: {SMOOTH_ROUTES_DIR}", file=sys.stderr)
        sys.exit(1)

    counts = count_trips_per_hour(SMOOTH_ROUTES_DIR)
    if not counts:
        print("[INFO] Không tìm thấy file hợp lệ hoặc không có vehicle nào.", file=sys.stderr)
        return

    print("Số chuyến (vehicles) theo từng giờ trong thư mục smooth-routes:")
    print("Hour\tCount")
    for hour, cnt in counts.items():
        print(f"{hour}\t{cnt}")


if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
merge_vehicle_routes.py
-----------------------
Merge tất cả các file routes_XX_YofZ.xml (liệt kê <vehicle> chi tiết)
thuộc cùng một giờ XX thành một file routes_XX_merged.xml có period=3600s.

Cách chạy:
    python3 merge_vehicle_routes.py

Sau khi chạy xong, bạn sẽ có thư mục:
    smooth-routes-merged/
        routes_00_merged.xml
        routes_01_merged.xml
        ...
        routes_23_merged.xml
(chỉ tạo các giờ thực sự có file con).
"""

import sys
import re
import xml.etree.ElementTree as ET
from pathlib import Path
from collections import defaultdict

# ---------------------------------------------------------------------------
# Thay đổi nếu cần:
INPUT_DIR = Path(
    "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/"
    "data/processed-data/total_scenario/smooth-routes"
)
OUTPUT_DIR = Path(
    "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/"
    "data/processed-data/total_scenario/smooth-routes-merged"
)
# Mỗi giờ = 3600s
HOUR = 3600

# Regex để bắt tên file: routes_XX_YofZ.xml (vd. routes_07_2of3.xml)
pattern = re.compile(r"^routes_(\d{2})_(\d+)of(\d+)\.xml$")

# Tạo thư mục output nếu chưa có
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------------------------------------------------------------------------
# Bước 1: Quét tất cả file *.xml trong INPUT_DIR, group theo giờ
# ---------------------------------------------------------------------------
hour_to_entries = defaultdict(list)
for f in INPUT_DIR.glob("*.xml"):
    if ".alt.xml" in f.name:
        continue
    m = pattern.match(f.name)
    if not m:
        continue
    hh = int(m.group(1))      # ví dụ "07" → 7
    i = int(m.group(2))       # ví dụ "2" (trong "2of3")
    m_val = int(m.group(3))   # ví dụ "3" (trong "2of3")
    hour_to_entries[hh].append((f, i, m_val))

if not hour_to_entries:
    print(f"[ERROR] Không tìm thấy file routes_XX_YofZ.xml trong {INPUT_DIR}")
    sys.exit(1)

# ---------------------------------------------------------------------------
# Bước 2: Với mỗi giờ, merge thành routes_XX_merged.xml
# ---------------------------------------------------------------------------
for hh in sorted(hour_to_entries):
    entries = hour_to_entries[hh]
    # Giả định mọi entry của cùng giờ đều có cùng "m" (of m)
    m_values = set(mv for (_, _, mv) in entries)
    if len(m_values) != 1:
        print(f"[WARNING] Giờ {hh:02d} có nhiều giá trị m khác nhau: {m_values}")
    m_of = sorted(m_values)[0]
    period_sub = HOUR // m_of  # 3600/m

    print(f"[INFO] Giờ {hh:02d}: có {len(entries)} file (of{m_of}), period_sub={period_sub}s.")

    # Tạo tree mới (root=<routes>)
    merged_root = ET.Element("routes")

    # 2.1. Duyệt một vòng để thu thập vType và route định nghĩa chung
    vtype_dict = {}
    route_dict = {}
    for (route_file, _, _) in sorted(entries, key=lambda x: x[1]):
        tree = ET.parse(route_file)
        root = tree.getroot()
        # vType
        for v in root.findall("vType"):
            vid = v.attrib.get("id")
            if vid and vid not in vtype_dict:
                vtype_dict[vid] = ET.fromstring(ET.tostring(v))
        # route
        for r in root.findall("route"):
            rid = r.attrib.get("id")
            if rid and rid not in route_dict:
                route_dict[rid] = ET.fromstring(ET.tostring(r))

    # Append vType và route vào merged_root
    for v in vtype_dict.values():
        merged_root.append(v)
    for r in route_dict.values():
        merged_root.append(r)

    # 2.2. Duyệt lại từng file con, adjust depart của vehicle, append vào merged_root
    for (route_file, idx, _) in sorted(entries, key=lambda x: x[1]):
        offset = (idx - 1) * period_sub
        tree = ET.parse(route_file)
        root = tree.getroot()
        for veh in root.findall("vehicle"):
            veh_copy = ET.fromstring(ET.tostring(veh))
            depart_old = float(veh_copy.attrib.get("depart", "0"))
            depart_new = depart_old + offset
            veh_copy.set("depart", f"{depart_new:.2f}")
            merged_root.append(veh_copy)

    # 2.3. Ghi ra file merged
    merged_tree = ET.ElementTree(merged_root)
    merged_name = f"routes_{hh:02d}_merged.xml"
    merged_path = OUTPUT_DIR / merged_name
    merged_tree.write(merged_path, encoding="utf-8", xml_declaration=True)
    print(f"    → Đã tạo {merged_name} chứa {len(merged_root.findall('vehicle'))} vehicles.")

print("=== Hoàn thành merge cho tất cả các giờ! ===")


In [2]:
#!/usr/bin/env python3
"""
create_tls_for_merged.py
------------------------
Sau khi đã merge routes vào folder "smooth-routes-merged", dùng script này
để:
 1. Với mỗi file routes_XX_merged.xml, sinh:
       - tls_cycle_XX.add.xml
       - tls_coord_XX.add.xml
    → Lưu ở thư mục "smooth-tls/base/".
 2. Sau đó, copy hai file base đó thành N cặp file tương ứng với of N.
    Ví dụ giờ 07 có of3, sẽ tạo:
       - tls_cycle_07_1of3.add.xml
       - tls_coord_07_1of3.add.xml
       - tls_cycle_07_2of3.add.xml
       - tls_coord_07_2of3.add.xml
       - tls_cycle_07_3of3.add.xml
       - tls_coord_07_3of3.add.xml
"""

import sys
import subprocess
from pathlib import Path
import re
import shutil

# ---------------------------------------------------------------------------
# Thay đổi đường dẫn nếu cần:
# ---------------------------------------------------------------------------
# Đường dẫn file mạng lưới SUMO
NET_XML = (
    "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/"
    "result/net-creation/310525-AMP-test-func/connected-network.net.xml"
)
# Đường dẫn tới tlsCycleAdaptation.py và tlsCoordinator.py
sys.path.append("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/src")
try:
    from config import TLSCYCLEADAPTATION_PY, TLSCOORDINATOR_PY  # noqa: E402
except ImportError:
    print("[ERROR] Không tìm thấy config.py hoặc TLSCYCLEADAPTATION_PY/TLSCOORDINATOR_PY chưa khai báo.")
    sys.exit(1)

# Folder chứa routes đã merge (period=3600)
MERGED_DIR = Path(
    "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/"
    "data/processed-data/total_scenario/smooth-routes-merged"
)

# Root folder chứa toàn bộ smooth-routes (để quét sub-of)
ORIG_ROUTES_DIR = MERGED_DIR.parent  # ví dụ: smooth-routes

# Folder đích lưu TLS (cuối cùng)
TLS_DIR = Path(
    "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/"
    "data/processed-data/total_scenario/smooth-tls"
)

# Folder tạm để lưu file base (chưa phân of N)
BASE_TLS_DIR = TLS_DIR / "base"
BASE_TLS_DIR.mkdir(parents=True, exist_ok=True)
TLS_DIR.mkdir(parents=True, exist_ok=True)

# Regex để bắt tên file merged: routes_XX_merged.xml
merged_pattern = re.compile(r"^routes_(\d{2})_merged\.xml$")

# ---------------------------------------------------------------------------
# Bước 1: Với mỗi routes_XX_merged.xml, chạy tlsCycleAdaptation + tlsCoordinator
# ---------------------------------------------------------------------------
for merged_file in sorted(MERGED_DIR.glob("routes_*_merged.xml")):
    m = merged_pattern.match(merged_file.name)
    if not m:
        continue
    hh = int(m.group(1))
    tag = f"{hh:02d}"
    begin_time = hh * 3600

    # Tên file base (chưa phân of)
    tls_cycle_base = BASE_TLS_DIR / f"tls_cycle_{tag}.add.xml"
    tls_coord_base = BASE_TLS_DIR / f"tls_coord_{tag}.add.xml"

    # 1. Tạo tls_cycle_base nếu chưa có hoặc file rỗng
    if not tls_cycle_base.exists() or tls_cycle_base.stat().st_size == 0:
        print(f"[{tag}] Tạo tls_cycle_base từ {merged_file.name} … (period=3600)")
        cmd_cycle = [
            "python3",
            TLSCYCLEADAPTATION_PY,
            "-n", NET_XML,
            "-r", str(merged_file),
            "-b", str(begin_time),
            "-o", str(tls_cycle_base),
            "--min-cycle", "40",
            "--max-cycle", "120",
            "--yellow-time", "3",
            "-p", "a",
            "--verbose",
        ]
        try:
            subprocess.run(cmd_cycle, check=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR] tlsCycleAdaptation thất bại cho giờ {tag}: {e}")
            continue
    else:
        print(f"[{tag}] tls_cycle_base đã tồn tại, bỏ qua.")

    # 2. Tạo tls_coord_base nếu chưa có hoặc file rỗng
    if not tls_coord_base.exists() or tls_coord_base.stat().st_size == 0:
        print(f"[{tag}] Tạo tls_coord_base từ {merged_file.name} …")
        cmd_coord = [
            "python3",
            TLSCOORDINATOR_PY,
            "-n", NET_XML,
            "-r", str(merged_file),
            "-a", str(tls_cycle_base),
            "-o", str(tls_coord_base),
        ]
        try:
            subprocess.run(cmd_coord, check=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR] tlsCoordinator thất bại cho giờ {tag}: {e}")
            continue
    else:
        print(f"[{tag}] tls_coord_base đã tồn tại, bỏ qua.")

print("=== Hoàn thành tạo base TLS cho mọi giờ! ===\n")

# ---------------------------------------------------------------------------
# Bước 2: Lấy thông tin số phần of N và copy base → of N
# ---------------------------------------------------------------------------

# Regex để bắt tên file gốc: routes_XX_partofM.xml
orig_pattern = re.compile(r"^routes_(\d{2})_(\d+)of(\d+)\.xml$")

# Tạo map: hour → max_m (giả sử mọi file cùng giờ có cùng M)
hour_to_m: dict[int, int] = {}
for f in sorted(ORIG_ROUTES_DIR.glob("routes_*.xml")):
    m = orig_pattern.match(f.name)
    if not m:
        continue
    hh = int(m.group(1))
    m_of = int(m.group(3))
    # Lấy max của m_of
    if hh not in hour_to_m or hour_to_m[hh] < m_of:
        hour_to_m[hh] = m_of

if not hour_to_m:
    print("[WARNING] Không tìm thấy bất kỳ file sub-of trong thư mục gốc routes.")
else:
    print(f"[INFO] Phát hiện cấu hình of M như sau: {hour_to_m}\n")

# Với mỗi giờ, copy base thành các ofN
for hh, m_of in sorted(hour_to_m.items()):
    tag = f"{hh:02d}"
    base_cycle = BASE_TLS_DIR / f"tls_cycle_{tag}.add.xml"
    base_coord = BASE_TLS_DIR / f"tls_coord_{tag}.add.xml"

    if not base_cycle.exists() or not base_coord.exists():
        print(f"[WARNING] Bỏ qua giờ {tag}, không có base TLS trong {BASE_TLS_DIR}.")
        continue

    print(f"[{tag}] Phân ra {m_of} file TLS tương ứng of{m_of}:")

    for part in range(1, m_of + 1):
        new_cycle = TLS_DIR / f"tls_cycle_{tag}_{part}of{m_of}.add.xml"
        new_coord = TLS_DIR / f"tls_coord_{tag}_{part}of{m_of}.add.xml"

        # Copy nếu chưa tồn tại hoặc muốn ghi đè thì remove rồi copy
        if new_cycle.exists():
            new_cycle.unlink()
        shutil.copy(base_cycle, new_cycle)
        print(f"  - Tạo {new_cycle.name}")

        if new_coord.exists():
            new_coord.unlink()
        shutil.copy(base_coord, new_coord)
        print(f"  - Tạo {new_coord.name}")

    print("")  # dòng trống giữa các giờ

print("=== Hoàn thành split base TLS thành các ofN! ===")


[00] tls_cycle_base đã tồn tại, bỏ qua.
[00] tls_coord_base đã tồn tại, bỏ qua.
[01] Tạo tls_cycle_base từ routes_01_merged.xml … (period=3600)
the total number of tls: 5960
Begin time:3600.0
parsing route file: /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-routes-merged/routes_01_merged.xml
Parsed 158619 vehicles passing traffic lights from /home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-routes-merged/routes_01_merged.xml.
tl-logic ID: 10004336247
phase: 0
group flows: [79, 17.00273888888889, 7.001127777777778]
The used lanes: [[0], [0]]
the current cycle length:90 sec
Green time for phase 0: 37
The optimal cycle length:40

Duration for Phase 1 is from the input file.
Duration for Phase 2 is from the input file.
tl-logic ID: 10004336274
phase: 0
group flows: [79, 17.00273888888889, 7.001127777777778]
The used lanes: [[0], [0]]
the current cycle length:90 sec
Green time for phase 0: 37
The optimal cycle length: